In [1]:
import argparse
import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from io import FileIO
import csv
import pandas as pd

In [2]:
from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

In [3]:
API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'

CLIENT_SECRETS_FILE = 'client_secret_558997714597-6tddji99vgbrka974vcnt430sfsdavso.apps.googleusercontent.com.json'

In [4]:
def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename='output.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

In [13]:
if __name__ == '__main__':
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    youtubeAnalytics = get_service()
    result = execute_api_request(
        youtubeAnalytics.reports().query,
        ids='channel==MINE',
        startDate='2022-01-01',
        endDate='2023-08-05',
        metrics='estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost',
        dimensions='day',
        sort='day'
    )
    headers = ['date', 'estMinutesWatched', 'views', 'likes', 'dislikes', 'subscribersGained', 'subscribersLost']#, 'comments', 'estimatedMinutesWatched', 'averageViewDuration', 'relativeRetentionPerformance']
    create_table(result['rows'], headers=headers)
    create_csv(result['rows'], headers=headers)
    print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))
    #print(result)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=558997714597-6tddji99vgbrka974vcnt430sfsdavso.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=6kT0EEMeLOBh44Fv9oMls4PPbQrvfZ&prompt=consent&access_type=offline
	date		estMinutesWatched		views		likes		dislikes		subscribersGained		subscribersLost	
	2022-01-01		2		3		0		0		1		0	
	2022-01-02		2		5		0		0		0		0	
	2022-01-03		2		6		0		0		0		0	
	2022-01-04		0		0		0		0		0		0	
	2022-01-05		0		1		0		0		0		0	
	2022-01-06		2		2		0		0		0		0	
	2022-01-07		1		3		0		0		0		0	
	2022-01-08		0		0		0		0		0		0	
	2022-01-09		0		1		0		0		0		0	
	2022-01-10		9		14		2		0		1		0	
	2022-01-11		17		23		1		0		0		0	
	2022-01-12		12		23		1		0		0		0	
	2022-01-13		12		19		13		0		2		0	
	2022-01-14		19		22		7		0		1		0	
	2022-01-15		58		89		3		0		1		0	
	2022-01-16		22		41		6		0		2		0	
	2022-01-17		8		